In [1]:
# importing necessary libraries
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.model_selection import cross_val_score

In [2]:
# exporting data
data = pd.read_csv('data.csv')
data.head()

,Cost Rank,Product Name,Product Life Cycle,FY20 Q4,FY21 Q1,FY21 Q2,FY21 Q3,FY21 Q4,FY22 Q1,FY22 Q2,FY22 Q3,FY22 Q4,FY23 Q1,FY23 Q2,FY23 Q3,FY23Q4 Demand,FYQ23 Q4 Marketing,FY23 Q4 Stats/ML
0,1,ROUTER 1,Sustaining,NaN,NaN,NaN,906.0,941.0,663.0,769.0,3565.0,1020,1670,38,2141,"1,716","1,716","1,948"
1,2,TRANSCIEVER,Sustaining,87173.0,102849.0,126375.0,128629.0,179125.0,138750.0,208760.0,116126.0,150803,82163,82408,67132,"85,000","141,285","97,995"
2,3,SWITCH 1,Sustaining,NaN,NaN,NaN,3651.0,10409.0,10575.0,17934.0,15104.0,15525,6938,9595,9231,"12,000","11,820","11,184"
3,4,ACCESS POINT 1,Sustaining,NaN,NaN,NaN,NaN,NaN,NaN,9576.0,46029.0,109452,54955,60478,62708,"62,771","79,113","65,310"
4,5,ACCESS POINT 2,Sustaining,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26769,33670,74853,80944,"76,192","113,704","93,200"


In [3]:
#removing unnecessary columns
data = data.drop(['Cost Rank','Product Life Cycle','FY23Q4 Demand','FYQ23 Q4 Marketing','FY23 Q4 Stats/ML'], axis = 1)
data.set_index('Product Name', inplace = True)
data.head()

,FY20 Q4,FY21 Q1,FY21 Q2,FY21 Q3,FY21 Q4,FY22 Q1,FY22 Q2,FY22 Q3,FY22 Q4,FY23 Q1,FY23 Q2,FY23 Q3
Product Name,,,,,,,,,,,,
ROUTER 1,NaN,NaN,NaN,906.0,941.0,663.0,769.0,3565.0,1020,1670,38,2141
TRANSCIEVER,87173.0,102849.0,126375.0,128629.0,179125.0,138750.0,208760.0,116126.0,150803,82163,82408,67132
SWITCH 1,NaN,NaN,NaN,3651.0,10409.0,10575.0,17934.0,15104.0,15525,6938,9595,9231
ACCESS POINT 1,NaN,NaN,NaN,NaN,NaN,NaN,9576.0,46029.0,109452,54955,60478,62708
ACCESS POINT 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26769,33670,74853,80944


In [4]:
data.shape

(20, 12)

In [5]:
transposed = data.transpose()
transposed.shape

(12, 20)

In [6]:
transposed

Product Name,ROUTER 1,TRANSCIEVER,SWITCH 1,ACCESS POINT 1,ACCESS POINT 2,SWITCH 2,SWITCH 3,POWER SUPPLY 1,SWITCH 4,SWITCH 5,SWITCH 6,ACCESS POINT 3,SUPERVISOR ENGINE,SWITCH 7,WIRELESS CONTROLLER,SWITCH 8,SWITCH 9,ACCESS POINT 4,SWITCH 10,POWER SUPPLY 2
FY20 Q4,NaN,87173.0,NaN,NaN,NaN,13432.0,28585.0,115709.0,NaN,4571.0,NaN,65048.0,NaN,6073.0,1208.0,4.0,6187.0,NaN,NaN,28279.0
FY21 Q1,NaN,102849.0,NaN,NaN,NaN,12463.0,19686.0,98337.0,NaN,4100.0,NaN,67810.0,NaN,5459.0,963.0,-4.0,5301.0,NaN,NaN,22704.0
FY21 Q2,NaN,126375.0,NaN,NaN,NaN,13452.0,44724.0,147681.0,NaN,4830.0,NaN,90928.0,NaN,6736.0,1698.0,36.0,7172.0,NaN,NaN,24098.0
FY21 Q3,906.0,128629.0,3651.0,NaN,NaN,14581.0,37781.0,185327.0,177.0,5472.0,NaN,148547.0,NaN,12213.0,1833.0,305.0,7582.0,NaN,NaN,31461.0
FY21 Q4,941.0,179125.0,10409.0,NaN,NaN,15831.0,31905.0,194170.0,270.0,7391.0,NaN,228945.0,NaN,13883.0,2423.0,1093.0,10964.0,NaN,NaN,47009.0
FY22 Q1,663.0,138750.0,10575.0,NaN,NaN,13864.0,37888.0,220386.0,850.0,7894.0,NaN,204184.0,NaN,27045.0,2602.0,2360.0,11501.0,NaN,NaN,35145.0
FY22 Q2,769.0,208760.0,17934.0,9576.0,NaN,15012.0,32689.0,204604.0,2717.0,7101.0,222.0,245453.0,15.0,15162.0,2341.0,4317.0,14980.0,NaN,NaN,45978.0
FY22 Q3,3565.0,116126.0,15104.0,46029.0,NaN,11349.0,30442.0,194538.0,3833.0,6943.0,1549.0,245803.0,108.0,13855.0,1830.0,4343.0,9158.0,NaN,NaN,34518.0
FY22 Q4,1020.0,150803.0,15525.0,109452.0,26769.0,10422.0,27028.0,185160.0,4279.0,6126.0,4619.0,136283.0,1972.0,12351.0,1567.0,4094.0,11601.0,13260.0,1227.0,40605.0
FY23 Q1,1670.0,82163.0,6938.0,54955.0,33670.0,8726.0,21246.0,147284.0,6089.0,4616.0,4764.0,76020.0,2427.0,8933.0,1210.0,2756.0,3788.0,21667.0,24186.0,26925.0


In [7]:
transposed.index

Index(['FY20 Q4', 'FY21 Q1', 'FY21 Q2', 'FY21 Q3', 'FY21 Q4', 'FY22 Q1',
       'FY22 Q2', 'FY22 Q3', 'FY22 Q4', 'FY23 Q1', 'FY23 Q2', 'FY23 Q3'],
      dtype='object')

In [8]:
transposed = transposed.reset_index().rename(columns={'index': 'Quarters'})


In [9]:
# transposed.index

In [10]:
# transposed.columns

In [11]:
transposed

Product Name,Quarters,ROUTER 1,TRANSCIEVER,SWITCH 1,ACCESS POINT 1,ACCESS POINT 2,SWITCH 2,SWITCH 3,POWER SUPPLY 1,SWITCH 4,...,SWITCH 6,ACCESS POINT 3,SUPERVISOR ENGINE,SWITCH 7,WIRELESS CONTROLLER,SWITCH 8,SWITCH 9,ACCESS POINT 4,SWITCH 10,POWER SUPPLY 2
0,FY20 Q4,NaN,87173.0,NaN,NaN,NaN,13432.0,28585.0,115709.0,NaN,...,NaN,65048.0,NaN,6073.0,1208.0,4.0,6187.0,NaN,NaN,28279.0
1,FY21 Q1,NaN,102849.0,NaN,NaN,NaN,12463.0,19686.0,98337.0,NaN,...,NaN,67810.0,NaN,5459.0,963.0,-4.0,5301.0,NaN,NaN,22704.0
2,FY21 Q2,NaN,126375.0,NaN,NaN,NaN,13452.0,44724.0,147681.0,NaN,...,NaN,90928.0,NaN,6736.0,1698.0,36.0,7172.0,NaN,NaN,24098.0
3,FY21 Q3,906.0,128629.0,3651.0,NaN,NaN,14581.0,37781.0,185327.0,177.0,...,NaN,148547.0,NaN,12213.0,1833.0,305.0,7582.0,NaN,NaN,31461.0
4,FY21 Q4,941.0,179125.0,10409.0,NaN,NaN,15831.0,31905.0,194170.0,270.0,...,NaN,228945.0,NaN,13883.0,2423.0,1093.0,10964.0,NaN,NaN,47009.0
5,FY22 Q1,663.0,138750.0,10575.0,NaN,NaN,13864.0,37888.0,220386.0,850.0,...,NaN,204184.0,NaN,27045.0,2602.0,2360.0,11501.0,NaN,NaN,35145.0
6,FY22 Q2,769.0,208760.0,17934.0,9576.0,NaN,15012.0,32689.0,204604.0,2717.0,...,222.0,245453.0,15.0,15162.0,2341.0,4317.0,14980.0,NaN,NaN,45978.0
7,FY22 Q3,3565.0,116126.0,15104.0,46029.0,NaN,11349.0,30442.0,194538.0,3833.0,...,1549.0,245803.0,108.0,13855.0,1830.0,4343.0,9158.0,NaN,NaN,34518.0
8,FY22 Q4,1020.0,150803.0,15525.0,109452.0,26769.0,10422.0,27028.0,185160.0,4279.0,...,4619.0,136283.0,1972.0,12351.0,1567.0,4094.0,11601.0,13260.0,1227.0,40605.0
9,FY23 Q1,1670.0,82163.0,6938.0,54955.0,33670.0,8726.0,21246.0,147284.0,6089.0,...,4764.0,76020.0,2427.0,8933.0,1210.0,2756.0,3788.0,21667.0,24186.0,26925.0


In [18]:
transposed['Quarters'] = ['2020 Q4','2021 Q1','2021 Q2','2021 Q3','2021 Q4','2022 Q1','2022 Q2','2022 Q3','2022 Q4','2023 Q1','2023 Q2','2023 Q3']

In [19]:
transposed

Product Name,Quarters,ROUTER 1,TRANSCIEVER,SWITCH 1,ACCESS POINT 1,ACCESS POINT 2,SWITCH 2,SWITCH 3,POWER SUPPLY 1,SWITCH 4,...,SWITCH 6,ACCESS POINT 3,SUPERVISOR ENGINE,SWITCH 7,WIRELESS CONTROLLER,SWITCH 8,SWITCH 9,ACCESS POINT 4,SWITCH 10,POWER SUPPLY 2
0,2020 Q4,NaN,87173.0,NaN,NaN,NaN,13432.0,28585.0,115709.0,NaN,...,NaN,65048.0,NaN,6073.0,1208.0,4.0,6187.0,NaN,NaN,28279.0
1,2021 Q1,NaN,102849.0,NaN,NaN,NaN,12463.0,19686.0,98337.0,NaN,...,NaN,67810.0,NaN,5459.0,963.0,-4.0,5301.0,NaN,NaN,22704.0
2,2021 Q2,NaN,126375.0,NaN,NaN,NaN,13452.0,44724.0,147681.0,NaN,...,NaN,90928.0,NaN,6736.0,1698.0,36.0,7172.0,NaN,NaN,24098.0
3,2021 Q3,906.0,128629.0,3651.0,NaN,NaN,14581.0,37781.0,185327.0,177.0,...,NaN,148547.0,NaN,12213.0,1833.0,305.0,7582.0,NaN,NaN,31461.0
4,2021 Q4,941.0,179125.0,10409.0,NaN,NaN,15831.0,31905.0,194170.0,270.0,...,NaN,228945.0,NaN,13883.0,2423.0,1093.0,10964.0,NaN,NaN,47009.0
5,2022 Q1,663.0,138750.0,10575.0,NaN,NaN,13864.0,37888.0,220386.0,850.0,...,NaN,204184.0,NaN,27045.0,2602.0,2360.0,11501.0,NaN,NaN,35145.0
6,2022 Q2,769.0,208760.0,17934.0,9576.0,NaN,15012.0,32689.0,204604.0,2717.0,...,222.0,245453.0,15.0,15162.0,2341.0,4317.0,14980.0,NaN,NaN,45978.0
7,2022 Q3,3565.0,116126.0,15104.0,46029.0,NaN,11349.0,30442.0,194538.0,3833.0,...,1549.0,245803.0,108.0,13855.0,1830.0,4343.0,9158.0,NaN,NaN,34518.0
8,2022 Q4,1020.0,150803.0,15525.0,109452.0,26769.0,10422.0,27028.0,185160.0,4279.0,...,4619.0,136283.0,1972.0,12351.0,1567.0,4094.0,11601.0,13260.0,1227.0,40605.0
9,2023 Q1,1670.0,82163.0,6938.0,54955.0,33670.0,8726.0,21246.0,147284.0,6089.0,...,4764.0,76020.0,2427.0,8933.0,1210.0,2756.0,3788.0,21667.0,24186.0,26925.0


In [20]:
# Split the 'Quarters' column into year and quarter
transposed[['Year', 'Quarter']] = transposed['Quarters'].str.split(' ', expand=True)

# Map quarter strings to their numeric representations
quarter_map = {'Q1': 1, 'Q2': 4, 'Q3': 7, 'Q4': 10}
transposed['Quarter'] = transposed['Quarter'].map(quarter_map)

# Convert to datetime
transposed['Date'] = pd.to_datetime(transposed['Year'] + '-' + transposed['Quarter'].astype(str), format='%Y-%m')

# Convert to period
transposed['Quarter'] = transposed['Date'].dt.to_period('Q')

In [21]:
transposed

Product Name,Quarters,ROUTER 1,TRANSCIEVER,SWITCH 1,ACCESS POINT 1,ACCESS POINT 2,SWITCH 2,SWITCH 3,POWER SUPPLY 1,SWITCH 4,...,SWITCH 7,WIRELESS CONTROLLER,SWITCH 8,SWITCH 9,ACCESS POINT 4,SWITCH 10,POWER SUPPLY 2,Year,Quarter,Date
0,2020 Q4,NaN,87173.0,NaN,NaN,NaN,13432.0,28585.0,115709.0,NaN,...,6073.0,1208.0,4.0,6187.0,NaN,NaN,28279.0,2020,2020Q4,2020-10-01
1,2021 Q1,NaN,102849.0,NaN,NaN,NaN,12463.0,19686.0,98337.0,NaN,...,5459.0,963.0,-4.0,5301.0,NaN,NaN,22704.0,2021,2021Q1,2021-01-01
2,2021 Q2,NaN,126375.0,NaN,NaN,NaN,13452.0,44724.0,147681.0,NaN,...,6736.0,1698.0,36.0,7172.0,NaN,NaN,24098.0,2021,2021Q2,2021-04-01
3,2021 Q3,906.0,128629.0,3651.0,NaN,NaN,14581.0,37781.0,185327.0,177.0,...,12213.0,1833.0,305.0,7582.0,NaN,NaN,31461.0,2021,2021Q3,2021-07-01
4,2021 Q4,941.0,179125.0,10409.0,NaN,NaN,15831.0,31905.0,194170.0,270.0,...,13883.0,2423.0,1093.0,10964.0,NaN,NaN,47009.0,2021,2021Q4,2021-10-01
5,2022 Q1,663.0,138750.0,10575.0,NaN,NaN,13864.0,37888.0,220386.0,850.0,...,27045.0,2602.0,2360.0,11501.0,NaN,NaN,35145.0,2022,2022Q1,2022-01-01
6,2022 Q2,769.0,208760.0,17934.0,9576.0,NaN,15012.0,32689.0,204604.0,2717.0,...,15162.0,2341.0,4317.0,14980.0,NaN,NaN,45978.0,2022,2022Q2,2022-04-01
7,2022 Q3,3565.0,116126.0,15104.0,46029.0,NaN,11349.0,30442.0,194538.0,3833.0,...,13855.0,1830.0,4343.0,9158.0,NaN,NaN,34518.0,2022,2022Q3,2022-07-01
8,2022 Q4,1020.0,150803.0,15525.0,109452.0,26769.0,10422.0,27028.0,185160.0,4279.0,...,12351.0,1567.0,4094.0,11601.0,13260.0,1227.0,40605.0,2022,2022Q4,2022-10-01
9,2023 Q1,1670.0,82163.0,6938.0,54955.0,33670.0,8726.0,21246.0,147284.0,6089.0,...,8933.0,1210.0,2756.0,3788.0,21667.0,24186.0,26925.0,2023,2023Q1,2023-01-01
